## MobileNet V2 model for training Food101 datasets
* number of parameters: 3,505,960

In [1]:
import torch

from torchvision import transforms, datasets

from pathlib import Path

import sys
sys.path.append('..')

from python_scripts import model_builder, engine, data_setup

In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

In [3]:
food101_transforms_train = transforms.Compose([
    transforms.RandomRotation(30),
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.AutoAugment(transforms.AutoAugmentPolicy.IMAGENET),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

food101_transforms_test = transforms.Compose([
    transforms.Resize(225),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
      [0.485, 0.456, 0.406],
      [0.229, 0.224, 0.225]
    )
])

In [4]:
learning_rate_list = [1e-4]
weight_decay_list = [1e-4]
epochs_list = [10]
batch_size_list = [16]

In [5]:
data_dir = Path('../data/extracted')

train_data_food101 = datasets.Food101(
    root=data_dir,
    split='train',
    transform=food101_transforms_train,
    download=True
)
test_data_food101 = datasets.Food101(
    root=data_dir,
    split='test',
    transform=food101_transforms_test,
    download=True
)

class_names_food101 = train_data_food101.classes

In [6]:
def model_generator(weights=None):
    return model_builder.MobileNetV2(num_classes=len(class_names_food101))

In [8]:
train_dataset, _ = data_setup.split_dataset(
    dataset=train_data_food101,
    split_size=1, # you can start with small number of train set to check normality at first
    seed=42
)
test_dataset, _ = data_setup.split_dataset(
    dataset=test_data_food101,
    split_size=1,
    seed=42
)

tuning_results = engine.HP_tune_train(
    model=None,
    model_generator=model_generator,
    model_weights=None,
    model_name='MobileNetV2_food101',
    train_dataset=train_dataset,
    test_dataset=test_dataset,
    learning_rate_list=learning_rate_list,
    weight_decay_list=weight_decay_list,
    epochs_list=epochs_list,
    batch_size_list=batch_size_list,
    is_tensorboard_writer=False,
    device=device,
    gradient_accumulation_num=1
)

[INFO] Splitting dataset of length 75750 into splits of size: 75750 and 0
[INFO] Splitting dataset of length 25250 into splits of size: 25250 and 0


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0 | Train_loss: 4.5331, Train_acc: 0.0250 | Test_loss: 4.1588, Test_acc: 0.0706
Epoch: 1 | Train_loss: 4.3420, Train_acc: 0.0478 | Test_loss: 3.9477, Test_acc: 0.0954
Epoch: 2 | Train_loss: 4.2236, Train_acc: 0.0664 | Test_loss: 3.7409, Test_acc: 0.1313
Epoch: 3 | Train_loss: 4.1058, Train_acc: 0.0838 | Test_loss: 3.5668, Test_acc: 0.1648
Epoch: 4 | Train_loss: 4.0022, Train_acc: 0.0990 | Test_loss: 3.3922, Test_acc: 0.1924
Epoch: 5 | Train_loss: 3.8841, Train_acc: 0.1216 | Test_loss: 3.1495, Test_acc: 0.2342
Epoch: 6 | Train_loss: 3.7768, Train_acc: 0.1389 | Test_loss: 3.0074, Test_acc: 0.2660
Epoch: 7 | Train_loss: 3.6593, Train_acc: 0.1611 | Test_loss: 2.8484, Test_acc: 0.3000
Epoch: 8 | Train_loss: 3.5592, Train_acc: 0.1804 | Test_loss: 2.6852, Test_acc: 0.3336
Epoch: 9 | Train_loss: 3.4571, Train_acc: 0.2000 | Test_loss: 2.5242, Test_acc: 0.3651
